In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io.wavfile import read
from scipy.signal import stft, get_window
import math

from music21 import *
import os

/Users/macbook/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [5]:
BPM = 60
oct4notes = { 
             "D4": 293,
             "E4": 329,
             "C4": 261,
             "G4": 392
             }

SAMPLING_RATE, signal = read('/Users/macbook/Fourier/mhll.wav')


# Convert BPM to approximate number of samples per beat
time_interval = 0.0625
#32nd note length
sample_count = time_interval * SAMPLING_RATE
resolution = 1/time_interval
# Ensure window size is appropriate (e.g., capped at 512 samples)
sample_count = 4000
print(resolution)
print(f"sample count: {sample_count}")
window = get_window("triang", sample_count)
print("Window type: ", type(window), " Window values: ", window)

16.0
sample count: 4000
Window type:  <class 'numpy.ndarray'>  Window values:  [0.00049975 0.0009995  0.00149925 ... 0.001999   0.00149925 0.0009995 ]


In [6]:
# Adjust STFT to scale frequency range to 0 - 1000 Hz
def short_time_fourier_transform(signal, overlap=0.25):  # Get the window for the STFT
    nperseg = len(window)
    noverlap = int(overlap * nperseg)
    f, t, Zxx = stft(signal, SAMPLING_RATE, window=window, nperseg=nperseg, noverlap=noverlap)
    return f, t, Zxx

In [ ]:
def getnotes():
    tolerance = 1.0  # Tolerance for frequency persistence in Hz
    frequency = 0
    count = 0
    
    for time_idx, time in enumerate(times):
        if time_idx == 0 or time_idx == len(times) - 1:
            continue
        
        #loop through timeindex and next time index and get all the amplitudes
        time_amplitudes_cur = amplitudes[:, time_idx]
        time_amplitudes_nxt = amplitudes[:, time_idx + 1]
        #get top two significant frequencies indicies
        top_indices_cur = np.argsort(time_amplitudes_cur)[-2:] 
        top_indices_nxt = np.argsort(time_amplitudes_nxt)[-2:]
        
        

        # Compare top frequencies based on indices and apply tolerance
        freq_cur = frequencies[top_indices_cur[1]]
        amp_cur = time_amplitudes_cur[top_indices_cur[1]]
        freq_nxt = frequencies[top_indices_nxt[1]]
        # ! prints indicies of top two freqs
        #print(f"Top indices (current): {top_indices_cur}, Top indices (next): {top_indices_nxt}")
        #  ! prints frequencies of top two
        #print(f"\n Frequencies (current): {frequencies[top_indices_cur]}, Frequencies (next): {frequencies[top_indices_nxt]}")
        # ! prints amplitud of top two
        # print(f"Amplitudes (current): {time_amplitudes_cur[top_indices_cur]}"

        # !prints the top four amplitudes
        print(f"Time: {time:.2f}s")
        top_indices = np.argsort(time_amplitudes_cur)[-4:]  # Get indices of top 4 amplitudes
        for idx in reversed(top_indices):  # Reverse to show the largest first
            print(f"  Frequency: {frequencies[idx]:.2f} Hz, Amplitude: {time_amplitudes_cur[idx]:.2f}")
        
        # ! if in consecutively in top 2 frequencies count increase
        # ! If not, get duration of the frequency, compare it to closest note, and append to list  
        #if freq_cur in frequencies[top_indices_nxt]:
        if amp_cur< 210:
            count +=1 
            length = get_duration(count)
            note = "rest"
        else:
            score.append((freq_cur,note,length,time))
            count = 0    
            if np.abs(freq_cur - freq_nxt) <= resolution:
                if amp_cur > 100: 
                    count += 1
            # ! detects when new note, or if same note repeated it splits by less amplitude frame
            elif count != 0:
                print(f" \n Frequency: {freq_cur:.2f} Hz gone, Amplitude: {amp_cur:.2f}, Count: {count}")
                length = get_duration(count)
                note = comparenotes(freq_cur)
                score.append((freq_cur,note,length,time))
                count = 0
            else: 
                pass

# Show data

In [ ]:
def Spectrogram(stft): 
    # Plot Frequency Spectrum (STFT Spectrogram) with Log Scaling
    plt.figure(figsize=(10, 6))
    plt.pcolormesh(times, frequencies, 10 * np.log10(np.abs(stft) + 1e-6), shading='gouraud', cmap='inferno')

    tick_positions = np.linspace(times[0], times[-1], num=24)  # More ticks

    plt.xticks(tick_positions, [f"{tick:.1f}" for tick in tick_positions])  # Custom labels
    plt.title(f"STFT Spectrogram, Sample count: {3000}, Precision: {3000/48000}ms", fontsize = 20)
    plt.xlabel("Time (s)")
    plt.ylabel("Frequency (Hz)")
    plt.colorbar(label="Amplitude (dB)")
    plt.grid()
    plt.tight_layout()
    plt.show()
    
def showSTFT(): 
    data = []
    for time_idx, time in enumerate(times):
        if time_idx == 0 or time_idx == len(times) - 1:
            continue
        
        #loop through timeindex and next time index and get all the amplitudes
        time_amplitudes_cur = amplitudes[:, time_idx]
        #get top two significant frequencies indicies
        top_indices_cur = np.argsort(time_amplitudes_cur)[-2:] 
        
        

        # Compare top frequencies based on indices and apply tolerance
        freq_cur = frequencies[top_indices_cur[1]]
        amp_cur = time_amplitudes_cur[top_indices_cur[1]]
        data[time_idx].append(time, freq_cur, amp_cur)
    
    # plot
    fig, ax = plt.subplots()
    for time, freq, amp in data:
        ax.scatter(time, freq, s=amp, c='blue', alpha=0.5)
    ax.set(xlim=(0, 8), xticks=np.arange(1, 8),
    ylim=(0, 8), yticks=np.arange(1, 8))
    plt.show()